# Import Library & Set Globals

In [ ]:
# base
import os
import sys
import warnings
import logging
logging.basicConfig(level='INFO')
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

# data manipulation
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
warnings.simplefilter("ignore", pd.errors.DtypeWarning)
mlogger = logging.getLogger('matplotlib')
mlogger.setLevel(logging.WARNING)

# single cell
import scanpy as sc
import decoupler as dc
from liana.method import cellphonedb

# custom
sys.path.append('../..')
from py import *

# R
import rpy2.robjects as ro
from rpy2.robjects.conversion import localconverter
converter = get_converter()
%load_ext rpy2.ipython

In [ ]:
%matplotlib inline
study = "paper_processed"
R_converted = False

CORES = 10
DATADIR = "../../../data"
REFDIR = "../../../references"
DOUBLETMETHODS = ['scDblFinder','DoubletFinder','doubletdetection','scrublet']

# Import Data

In [ ]:
study = "paper_processed"

# experiment = os.path.join('cellbender',study)
# filename = "output_filtered"
# filetype = ".h5"
# data_dir = DATADIR

experiment_dir = os.path.join('cellranger',study)
filename = "1/outs/raw_feature_bc_matrix"
filetype = ".mtx"
data_dir = DATADIR
convertR = False

adatas = get_data(data_dir, experiment_dir, filename, filetype, convertR)

for ad in adatas:
    ad.obs['Groups'] = ad.obs['Identifier']
    ad.obs['Experiment'], ad.obs['Condition'], ad.obs['Sample Type'] = np.array(ad.obs['Identifier'].str.split('-|_').to_list()).T

In [ ]:
def GetSpatialDataDIRECT(directory: str, name: str, filetype: str):
    if filetype == ".h5":
        adata = sc.read_10x_h5(directory + filetype)
    elif filetype == ".h5ad":
        adata = sc.read_h5ad(directory + filetype)
    elif filetype == ".mtx":
        adata = sc.read_10x_mtx(directory)
    else:
        raise ValueError(
            f"{filetype} is not a valid filetype. Must be h5, h5ad, or mtx."
        )

    adata.obs["Identifier"] = name
    adata.obs.index = adata.obs.index + "_" + name
    adata.layers["counts"] = adata.X.copy()

    return adata

def GetSpatialData(directory: str, name: str, filetype: str):
    if filetype == ".h5":
        adata = sc.read_10x_h5(directory + filetype)
    elif filetype == ".h5ad":
        adata = sc.read_h5ad(directory + filetype)
    elif filetype == ".mtx":
        adata = sc.read_10x_mtx(directory)
    else:
        raise ValueError(
            f"{filetype} is not a valid filetype. Must be h5, h5ad, or mtx."
        )

    adata.obs["Identifier"] = name
    adata.obs.index = adata.obs.index + "_" + name
    adata.layers["counts"] = adata.X.copy()

    return adata

adatas = get_data(data_dir, experiment_dir, filename, filetype, convertR)